In [0]:
%pip install fastf1 matplotlib pandas

In [0]:
%restart_python

In [0]:
# Notebook: 04_ingest_results
# Language: Python
# Markdown: small description
# MAGIC %md
# MAGIC # Ingest F1 Session Results into Bronze Layer
# MAGIC This notebook extracts session results (drivers, teams, finishing positions, points, etc.) and stores in bronze layer.

import fastf1, pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

CACHE_PATH = "/Workspace/Users/niranjan.482000@gmail.com/F1-Race-Analytics/cache"
try:
    fastf1.Cache.enable_cache(CACHE_PATH)
except Exception as e:
    print("Cache enable warning:", e)

# Example session
year = 2023
gp = "Bahrain"
sess_type = "R"

session = fastf1.get_session(year, gp, sess_type)
session.load()

# extract results
try:
    results = session.results.copy()  # Pandas DataFrame
except Exception as e:
    print("Results extraction failed:", e)
    results = pd.DataFrame()  # fallback empty

if not results.empty:
    results['event_name'] = session.event.EventName if hasattr(session.event, "EventName") else gp
    results['year'] = year
    results['round'] = getattr(session.event, "RoundNumber", None)
    results['session_name'] = session.name
    results['session_key'] = f"{results['event_name'].iloc[0]}_{session.name}_{pd.to_datetime(session.date).strftime('%Y%m%d')}"

    # create Spark DataFrame
    sdf = spark.createDataFrame(results.astype(str))  # convert all to string to avoid schema issues
    sdf.write.format("delta").mode("append").saveAsTable("f1_catalog.bronze.session_results")

    print(f"✅ Wrote {len(results)} rows for {results['session_key'].iloc[0]}")
    display(spark.table("f1_catalog.bronze.session_results").where(f"session_key = '{results['session_key'].iloc[0]}'").limit(20))
else:
    print("⚠️ No results available for this session")
